In [1]:
import re
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [5]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).collect()

In [6]:
def hledej(text, n=3, delka=200, frejm=df, sestupne=False, url=True):
    nalezy = frejm.sort(by="datum", descending=sestupne).filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        if url == True:
            print(f'URL: https://www.psp.cz/eknih/{radek[1].replace('_','/')}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [9]:
hledej("11. září")

Kdo: Předseda (zvoní)
Kde: 1918ns_ps_stenprot_070schuz_s070007.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/070schuz/s070007.htm
Kdy: 1919-09-10
Co: ch zpráv podati Nár. shromáždění další referát a eventuelně i případné návrhy. (Výborně! Potlesk.) Předseda: Přistoupím k ukončení schůze a navrhuji, aby příští schůze konala se zítra, ve čtvrtek dne 11. září 1919 o 10. hod. dop. s tímto denním pořadem: 1. Pokračování v rozpravě o naléhavých dotazech členů Nár. shromáždění Bechyně, Bradáče a soudr. o situaci na poli výroby, dopravy, výživy a sociální péče
***
Kdo: Zpravodaj posl. Zeminová
Kde: 1918ns_ps_stenprot_083schuz_s083003.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/083schuz/s083003.htm
Kdy: 1919-10-14
Co: o bude s letošní sklizní bramborovou. Zásobovací výbor o těchto věcech jednal ve dvou schůzích velmi podrobně a uvažovány všecky možné okolnosti, které mluví pro i proti státnímu obhospodařování. Dne 11. září konečně po bedlivém šetření usnesl se většinou hlasů výb

In [11]:
hledej("vítězný únor")

Kdo: Zpravodaj posl. dr. Bartuška
Kde: 1948ns_stenprot_020schuz_s020002.htm
URL: https://www.psp.cz/eknih/1948ns/stenprot/020schuz/s020002.htm
Kdy: 1948-12-21
Co: o shromáždění, který se na cestě k socialismu čestně přiřadí ke schválení naší ústavy 9. května. V tuto chvíli vrcholí staletý zápas našich národů o krajské zřízení. Tu je třeba podtrhnout, že teprve vítězný únor 1948 odstranil překážky, které reakční živly stavěly v cestu přestavbě státního aparátu podle zásad lidové demokracie a podle požadavků moderní, účelně organisované a decentralisované správy. Sleduje
***
Kdo: Zpravodajka posl. Kleňhová-Besserová
Kde: 1948ns_stenprot_020schuz_s020012.htm
URL: https://www.psp.cz/eknih/1948ns/stenprot/020schuz/s020012.htm
Kdy: 1948-12-21
Co: li, že jen ona jim může pomoci ke splnění jejich požadavků. Celá řada jich byla organisována v KSČ již před válkou a byla aktivní složkou dělnického hnutí. Musel však přijít květen 1945, musel přijít vítězný únor, aby se konečně i jim upravily praco

In [19]:
hledej(r"(?i)světov\w{0,10} obch\w{0,10} centr[aue]")

Kdo: Poslanec Miloslav Ransdorf
Kde: 2002ps_stenprot_014schuz_s014168.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/014schuz/s014168.htm
Kdy: 2003-04-15
Co: ují tento nový přístup k mezinárodnímu právu, kdy suverenita se ocitá v postavení pouhé chiméry, na celou zeměkouli. Já bych připomněl to, že už 13. září 2001, pouhé dva dny poté, co došlo k útoku na světové obchodní centrum, prohlásil Paul Wolfowitz, náměstek ministra obrany, tajemníka pro obranu Spojených států, že je třeba změnit charakter režimu v Iráku. A samozřejmě vytrvale za touto myšlenkou šel. Nyní tentýž Paul
***
Kdo: Senátorka Hana Žáková
Kde: 2021ps_stenprot_092schuz_s092024.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/092schuz/s092024.htm
Kdy: 2024-02-06
Co: oce 1995 je zprovozněn v areálu Jaderné elektrárny Dukovany první mezisklad použitého paliva, který je potom rozšířen v roce 2006 ještě o druhý, ale přichází 11. září roku 2001 a teroristický útok na Světové obchodní centrum v New Yorku, a je všem jasné, 

In [21]:
hledej(r"(?i) bin l[aá]d")

Kdo: Ministr zahraničních věcí ČR Jan Kavan
Kde: 1998ps_stenprot_022schuz_s022121.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/022schuz/s022121.htm
Kdy: 2000-02-24
Co: své rezoluci č. 1267 z 15. října loňského roku afghánské hnutí Talibán k zastavení poskytování podpory a zázemí mezinárodním teroristickým skupinám a k vydání jednoho z nejobávanějších teroristů Usámí bin Ladína do USA či jiné země, kde by byl souzen. Jednomyslné schválení rezoluce, kterou v Radě bezpečnosti podpořily rovněž muslimské státy, je mj. odrazem rostoucí obavy mezinárodního společenství z šířen
***
Kdo: Místopředsedkyně PSP Petra Buzková
Kde: 1998ps_stenprot_022schuz_s022123.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/022schuz/s022123.htm
Kdy: 2000-02-24
Co: ednotit postup, jak vytvořit určitý mezinárodněprávní úzus, který by nebyl napadnutelný a který by neměl statut výjimečnosti, jako je tomu v tomto případě. Jestliže se hovoří o hnutí Talibán a o osobě bin Ladína, tak úder, který byl s odvoláním 

In [27]:
hledej(r"(?i) obama")

Kdo: Předseda vlády ČR Mirek Topolánek
Kde: 2006ps_stenprot_021schuz_s021078.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/021schuz/s021078.htm
Kdy: 2007-09-26
Co: i. Myslím si, že návštěva významné představitelky amerického Kongresu za Demokratickou stranu paní Tauscher jednoznačně ukázala, že nemůžete očekávat, že po prezidentských volbách, které by vyhrál pan Obama nebo paní Hillary Clinton, by se změnila pozice Spojených států amerických v Missile Defense.
***
Kdo: Místopředseda vlády a ministr životního prostředí ČR Martin Bursík
Kde: 2006ps_stenprot_031schuz_s031008.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/031schuz/s031008.htm
Kdy: 2008-04-30
Co: , která bude hovořit s novou administrativou Spojených států o změně politiky ochrany klimatu ve Spojených státech, kterou lze očekávat, ať už bude ve Spojených státech zvolena za prezidentku Clinton, Obama nebo McCain, a bude tudíž rozhodující, nakolik bude české předsednictví úspěšné ve vztahu ke koordinaci společné klimat

In [29]:
hledej(r"Mnichov")

Kdo: Poslanec Dyk
Kde: 1918ns_ps_stenprot_020schuz_s020008.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/020schuz/s020008.htm
Kdy: 1919-01-23
Co: ů vznáším dotaz. Pokládáte to za zjev se stanoviska socialistického zdravý, je-li dnes ve vašich řadách tolik lidí, které tam nevedla ani zásada, ani myšlenka? Pokládáte to za zjev zdravý, jestliže v Mnichově za několik dní počet lidí, hlasujících pro socialistického kandidáta, se změnil o 19.000? Není to jakési memento, že ve vašich řadách jest ohromná řada přeběhlíků, lidí nespolehlivých? (Posl. Vaněk:
***
Kdo: Posl. prof. dr. Srdínko
Kde: 1918ns_ps_stenprot_034schuz_s034004.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/034schuz/s034004.htm
Kdy: 1919-02-27
Co: ut de France"; za přítomnosti Poincaréa a Ribota. protestovala proti ukrutnostem, které v Belgii a severní Francii Němci páší. Marně proti tomu ve spise "Válka a věda" bojuje president akademie věd v Mnichově, Heigel. Kdybychom chtěli s našeho hlediska posuzovati činy Ně

In [31]:
hledej(r"Maastri")

Kdo: Poslanec Josef Ježek
Kde: 1993ps_stenprot_037schuz_s037012.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/037schuz/s037012.htm
Kdy: 1995-12-06
Co: niční je jedním z nejdůležitějších bodů těchto debat. Např. na konferenci o regionálním plánování velké Evropy, konané zhruba před měsícem v Praze v hotelu Atrium Hilton, se hovořilo o tom, zda by do Maastrichtské dohody nemělo být doplněno označení onoho principu soudržnosti, tak důležitého, o dimenzi také regionální a územní. Tento princip subsidiality lze charakterizovat obecně asi takto. Jde o dosaže
***
Kdo: Poslanec Jiří Šoler
Kde: 1993ps_stenprot_039schuz_s039022.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/039schuz/s039022.htm
Kdy: 1996-02-08
Co: opská rada zřídila na summitu v červnu 1994 reflektní skupinu pro přípravu mezinárodní konference. Paní Elisabeth Guigou, členka reflektní skupiny, chce: odhodit byzantský legislativní proces zřízený Maastrichtem. Evropský parlament by měl být alespoň spolulegislativcem. Je na k

In [33]:
hledej(r"Scheng")

Kdo: Ministr vnitra ČR Jan Ruml
Kde: 1993ps_stenprot_025schuz_s025063.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/025schuz/s025063.htm
Kdy: 1994-12-15
Co: ké skupiny v Záhřebu v roce 1995. Tato skupina je tvořena z expertů následujících vlád: Maďarsko, které předsedá této komisi, Česká republika, Turecko, Chorvatsko, Evropská unie zastupovaná Německem, Schengenská skupina zastupovaná Německem. Práce této skupiny bude podporována třemi mezinárodními institucemi, Mezinárodní organizací pro migraci, Mezivládní konzultací a Mezinárodním centrem pro vývoj migr
***
Kdo: Poslanec Jaroslav Štrait
Kde: 1996ps_stenprot_017schuz_s017019.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/017schuz/s017019.htm
Kdy: 1997-12-04
Co: nchronizaci naší zahraniční politiky v zásadních otázkách a rozhodnutích při vstupu do nadnárodních útvarů. Nedovedu si dobře představit, že na hranicích česko-slovenských začnou platit v budoucnosti Schengenské dohody. Nedovedu si také představit, že si budeme desetil

In [39]:
hledej(r"eurobank")

Kdo: Poslanec Adolf Beznoska
Kde: 2013ps_stenprot_047schuz_s047166.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/047schuz/s047166.htm
Kdy: 2016-06-01
Co: Poslední se týká provozování anonymních zařízení a neanonymních zařízení přijímajících eurobankovky nebo euromince.
***


In [41]:
hledej(r" prask\w{0,10} bubl")

Kdo: Místopředseda PSP Vojtěch Filip
Kde: 2006ps_stenprot_031schuz_s031007.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/031schuz/s031007.htm
Kdy: 2008-04-30
Co: e žel, většinou za hranicemi naší země, tam, kde už není jenom jeden vlastník českých sdělovacích prostředků nebo dva vlastníci sdělovacích prostředků z jedné země, ty bubliny praskají. Připomenu, jak praskla bublina, jak jsme skvělí v zahraniční politice, když nás vypraskali na Valném shromáždění OSN jako nestálého člena Rady bezpečnosti. Znovu opakuji, místo České republiky Evropu zastupuje Chorvatsko, a oml
***
Kdo: Poslanec Vojtěch Filip
Kde: 2010ps_stenprot_032schuz_s032078.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/032schuz/s032078.htm
Kdy: 2011-12-08
Co:  finanční krizi a ten propad. Víte, to by ten finanční kapitál musel vůbec ten průmyslový kapitál, když už by šlo jenom o tyto kategorie, vůbec připustit k tomu, že se hodnotí stejným způsobem. Ale ty prasklé bubliny od roku 2007 přece jsou jasným důkazem, že